In [27]:
import csv
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
import os

In [28]:
def get_adjacent_matrix(distance_file: str,
                        num_nodes: int,
                        id_file: str = None,
                        graph_type="connect") -> np.array:
    """
    construct adjacent matrix by csv file   根据PEMS数据集的csv文件来构建邻接矩阵
    :param distance_file: path of csv file to save the distances between nodes  # csv文件路径
    :param num_nodes: number of nodes in the graph   graph中节点个数
    :param id_file: path of txt file to save the order of the nodes     
    :param graph_type: ["connect", "distance"] if use weight, please set distance
    :return:
    """
    A = np.zeros([int(num_nodes), int(num_nodes)])

    if id_file:
        with open(id_file, "r") as f_id:
            node_id_dict = {
                int(node_id): idx
                for idx, node_id in enumerate(f_id.read().strip().split("\n"))
            }

            with open(distance_file, "r") as f_d:
                f_d.readline()
                reader = csv.reader(f_d)
                for item in reader:
                    if len(item) != 3:
                        continue
                    i, j, distance = int(item[0]), int(item[1]), float(item[2])
                    if graph_type == "connect":
                        A[node_id_dict[i], node_id_dict[j]] = 1.
                        A[node_id_dict[j], node_id_dict[i]] = 1.
                    elif graph_type == "distance":
                        A[node_id_dict[i], node_id_dict[j]] = 1. / distance
                        A[node_id_dict[j], node_id_dict[i]] = 1. / distance
                    else:
                        raise ValueError(
                            "graph type is not correct (connect or distance)")
        return A

    with open(distance_file, "r") as f_d:
        f_d.readline()
        reader = csv.reader(f_d)
        for item in reader:
            if len(item) != 3:
                continue
            i, j, distance = int(item[0]), int(item[1]), float(item[2])

            if graph_type == "connect":
                A[i, j], A[j, i] = 1., 1.
            elif graph_type == "distance":
                A[i, j] = 1. / distance
                A[j, i] = 1. / distance
            else:
                raise ValueError(
                    "graph type is not correct (connect or distance)")

    return A


def get_flow_data(flow_file: str) -> np.array:
    """
    parse npz to get flow data  读取npz文件得到交通流数据
    :param flow_file: (N, T, D)
    :return:
    """
    data = np.load(flow_file)
    flow_data = data['data'].transpose([1, 0,
                                        2])[:, :,
                                            0][:, :,
                                               np.newaxis]  # [N, T, D]  D = 1
    return flow_data


class PEMSDataset(Dataset):

    def __init__(self, data_path, num_nodes, divide_days, time_interval,
                 history_length, train_mode):
        """
        load processed data
        :param data_path: ["graph file name" , "flow data file name"], path to save the data file names
        :param num_nodes: number of nodes in graph
        :param divide_days: [ days of train data, days of test data], list to divide the original data
        :param time_interval: time interval between two traffic data records (mins)
        :param history_length: length of history data to be used
        :param train_mode: ["train", "test"]
        """
        self.data_path = data_path
        self.num_nodes = num_nodes
        self.train_mode = train_mode
        self.train_days = divide_days[0]
        self.test_days = divide_days[1]
        self.history_length = history_length  # 6
        self.time_interval = time_interval  # 5 min
        self.one_day_length = int(24 * 60 / self.time_interval)
        self.graph = get_adjacent_matrix(distance_file=data_path[0],
                                         num_nodes=num_nodes)
        self.flow_norm, self.flow_data = self.pre_process_data(
            data=get_flow_data(data_path[1]), norm_dim=1)

    def __len__(self):
        if self.train_mode == "train":
            # return self.train_days * self.one_day_length - self.history_length  # 这里为什么要减掉一个history length，导致最后训练数据的长度为45*24*12-6=12954
            return self.train_days * self.one_day_length  # 这里为什么要减掉一个history length，导致最后训练数据的长度为45*24*12-6=12954
        elif self.train_mode == "test":
            return self.test_days * self.one_day_length
        else:
            raise ValueError("train mode: [{}] is not defined".format(
                self.train_mode))

    def __getitem__(self, index):  # (x, y), index = [0, L1 - 1]
        if self.train_mode == "train":
            index = index
        elif self.train_mode == "test":
            index += self.train_days * self.one_day_length
        else:
            raise ValueError("train mode: [{}] is not defined".format(
                self.train_mode))

        data_x, data_y = PEMSDataset.slice_data(self.flow_data,
                                                self.history_length, index,
                                                self.train_mode)
        data_x = PEMSDataset.to_tensor(data_x)  # [N, H, D]
        data_y = PEMSDataset.to_tensor(data_y).unsqueeze(1)  # [N, 1, D]
        return {
            "graph": PEMSDataset.to_tensor(self.graph),
            "flow_x": data_x,
            "flow_y": data_y
        }

    @staticmethod
    def slice_data(data, history_length, index, train_mode):
        """
        :param data: np.array, normalized traffic data.
        :param history_length: int, length of history data to be used.
        :param index: int, index on temporal axis.
        :param train_mode: str, ["train", "test"].
        :return:
            data_x: np.array, [N, H, D].
            data_y: np.array [N, D].
        """
        if train_mode == "train":
            start_index = index
            end_index = index + history_length
        elif train_mode == "test":
            start_index = index - history_length
            end_index = index
        else:
            raise ValueError(
                "train model {} is not defined".format(train_mode))

        data_x = data[:, start_index:end_index]
        data_y = data[:, end_index]

        return data_x, data_y

    @staticmethod
    def pre_process_data(data, norm_dim):
        """
        :param data: np.array, original traffic data without normalization.
        :param norm_dim: int, normalization dimension.
        :return:
            norm_base: list, [max_data, min_data], data of normalization base.
            norm_data: np.array, normalized traffic data.
        """
        norm_base = PEMSDataset.normalize_base(
            data, norm_dim)  # find the normalize base
        norm_data = PEMSDataset.normalize_data(norm_base[0], norm_base[1],
                                               data)  # normalize data

        return norm_base, norm_data

    @staticmethod
    def normalize_base(data, norm_dim):
        """
        :param data: np.array, original traffic data without normalization.
        :param norm_dim: int, normalization dimension.
        :return:
            max_data: np.array
            min_data: np.array
        """
        max_data = np.max(data, norm_dim,
                          keepdims=True)  # [N, T, D] , norm_dim=1, [N, 1, D]
        min_data = np.min(data, norm_dim, keepdims=True)
        return max_data, min_data

    @staticmethod
    def normalize_data(max_data, min_data, data):
        """
        :param max_data: np.array, max data.
        :param min_data: np.array, min data.
        :param data: np.array, original traffic data without normalization.
        :return:
            np.array, normalized traffic data.
        """
        mid = min_data
        base = max_data - min_data
        normalized_data = (data - mid) / base

        return normalized_data

    @staticmethod
    def recover_data(max_data, min_data, data):
        """
        :param max_data: np.array, max data.
        :param min_data: np.array, min data.
        :param data: np.array, normalized data.
        :return:
            recovered_data: np.array, recovered data.
        """
        mid = min_data
        base = max_data - min_data

        recovered_data = data * base + mid

        return recovered_data

    @staticmethod
    def to_tensor(data):
        return torch.tensor(data, dtype=torch.float)

In [29]:
def get_flow(filename):
    flow_data = np.load(filename)
    # print(type(flow_data))
    # PEMS数据集的data.npz文件中的data这个文件存放数据
    return flow_data['data']

def read_dataset(dataset):
    data = np.load(dataset)
    print(type(data['data']))
    print(data['data'][0].shape)

In [30]:
def get_loader(ds_name="PEMS04"):
    num_nodes = 307 if ds_name == 'PEMS04' else 170
    train_data = PEMSDataset(data_path=[
        "/home/zhoujianping/Research/GNN/TrafficPrediction/datasets/{}/distance.csv".format(ds_name),
        "/home/zhoujianping/Research/GNN/TrafficPrediction/datasets/{}/data.npz".format(ds_name)
    ],
                             num_nodes=num_nodes,
                             divide_days=[45, 14],
                             time_interval=5,
                             history_length=6,
                             train_mode="train")

    print(type(train_data))
    print(len(train_data))
    print(type(train_data[0]))
    print(train_data[0]['graph'].shape)
    print(train_data[0]['flow_x'].shape)
    print(train_data[0]['flow_y'].shape)
    
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    test_data = PEMSDataset(data_path=[
        "/home/zhoujianping/Research/GNN/TrafficPrediction/datasets/{}/distance.csv".format(ds_name),
        "/home/zhoujianping/Research/GNN/TrafficPrediction/datasets/{}/data.npz".format(ds_name)
    ],
                            num_nodes=num_nodes,
                            divide_days=[45, 14],
                            time_interval=5,
                            history_length=6,
                            train_mode="test")

    print(len(test_data))
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
    return train_loader, test_loader
    
train_loader, test_loader = get_loader('PEMS04')


<class '__main__.PEMSDataset'>
12960
<class 'dict'>
torch.Size([307, 307])
torch.Size([307, 6, 1])
torch.Size([307, 1, 1])
4032


In [31]:
for data in train_loader:
    print(data['flow_x'].shape)
    print(torch.squeeze(data['flow_x'],dim=3).shape)
    break

torch.Size([64, 307, 6, 1])
torch.Size([64, 307, 6])


In [32]:
# 超参数设置
# Hyper-parameters
image_size = 1842    # mnist数据集中一张图片的size，28*28
h_dim = 400 
z_dim = 20  
num_epochs = 10 # 迭代次数
batch_size = 64    # 一批样本的数量
learning_rate = 1e-3    # 学习率

In [33]:
# VAE model
class VAE(nn.Module):

    def __init__(self, image_size=1842, h_dim=400, z_dim=20):
        # 调用父类方法初始化模块的state
        super(VAE, self).__init__()

        # 编码器： [b,input_dim]-->[b,z_dim]
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)  # 均值 向量
        self.fc3 = nn.Linear(h_dim, z_dim)  # 保准方差 向量

        # 解码器：[b,z_dim]-->[b,input_dim]
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)

    # 编码过程
    def encode(self, x):
        """
        encoding part
        :param x: input image
        :return: mu and log_var
        """
        h = F.relu(self.fc1(x))
        mu = self.fc2(h)
        log_var = self.fc3(h)
        # return self.fc2(h), self.fc3(h)
        return mu, log_var

    # 随机生成隐含向量
    def reparameterize(self, mu, log_var):
        """
        Given a standard gaussian distribution epsilon ~ N(0,1),
        we can sample the random variable z as per z = mu + sigma * epsilon
        :param mu:
        :param log_var:
        :return: sampled z
        """
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    # 解码过程
    def decode(self, z):
        """
        Given a sampled z, decode it back to image
        :param z:
        :return:
        """
        h = F.relu(self.fc4(z))
        x_reconst = F.sigmoid(self.fc5(h))
        # return F.sigmoid(self.fc5(h))
        return x_reconst

    # 整个前向传播过程：编码-》解码
    def forward(self, x):
        """
        向前传播部分, 在model_name(inputs)时自动调用
        :param x: the input of our training model [b, batch_size, 1, 28, 28]
        :return: the result of our training model
        """
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

In [34]:
# 设备配置
torch.cuda.set_device(0) # 这句用来设置pytorch在哪块GPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
# 实例化一个模型
model = VAE().to(device)

# 创建优化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [38]:
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        # 获取样本，并前向传播
        x=torch.squeeze(data['flow_x'],dim=3)
        x = x.to(device).view(-1, image_size)   # 转换成二维（128,784）
        x_reconst, mu, log_var = model(x)

        # 计算重构损失和KL散度（KL散度用于衡量两种分布的相似程度）
        # KL散度的计算可以参考论文或者文章开头的链接
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        # 反向传播和优化
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(
                "Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}"
                .format(epoch + 1, num_epochs, i + 1, len(train_loader),
                        reconst_loss.item(), kl_div.item()))
    # print(x_reconst.shape)  # VAE输出的vector维度是(96,784)

Epoch[1/10], Step [100/203], Reconst Loss: 61268.6562, KL Div: 758.0039
Epoch[1/10], Step [200/203], Reconst Loss: 64866.1016, KL Div: 642.5427
Epoch[2/10], Step [100/203], Reconst Loss: 63678.6523, KL Div: 611.8344
Epoch[2/10], Step [200/203], Reconst Loss: 60216.4297, KL Div: 580.6030
Epoch[3/10], Step [100/203], Reconst Loss: 61349.4297, KL Div: 567.4918
Epoch[3/10], Step [200/203], Reconst Loss: 66332.0859, KL Div: 517.0936
Epoch[4/10], Step [100/203], Reconst Loss: 62164.1797, KL Div: 497.1253
Epoch[4/10], Step [200/203], Reconst Loss: 61858.0938, KL Div: 488.9102
Epoch[5/10], Step [100/203], Reconst Loss: 61873.6484, KL Div: 472.7031
Epoch[5/10], Step [200/203], Reconst Loss: 62857.2656, KL Div: 466.8595
Epoch[6/10], Step [100/203], Reconst Loss: 62506.0000, KL Div: 472.3004
Epoch[6/10], Step [200/203], Reconst Loss: 64931.8047, KL Div: 476.4633
Epoch[7/10], Step [100/203], Reconst Loss: 62782.0977, KL Div: 463.4772
Epoch[7/10], Step [200/203], Reconst Loss: 64430.5430, KL Div: 4